In [2]:
#@title Impressão da Matriz da Equação ARX com os Dados da Planilha
import pandas as pd
class Vetor:
    def __init__(self, tamanho):
        self.dados = [0.0] * tamanho
        self.tamanho = tamanho

    def inserir(self, i, valor):
        if 0 <= i < self.tamanho:
            self.dados[i] = valor
        else:
            raise IndexError("Índice fora dos limites.")

    def obter(self, i):
        if 0 <= i < self.tamanho:
            return self.dados[i]
        else:
            raise IndexError("Índice fora dos limites.")

    def como_lista(self):
        return self.dados.copy()


# Leitura da planilha

link = "https://docs.google.com/spreadsheets/d/1UPq-_KwH0DZXkwSryfIc5ePeHTOPsQVSuMVaHeimwSI/edit#gid=0"
csv_link = link.replace("/edit#gid=", "/export?format=csv&gid=")
dados = pd.read_csv(csv_link, decimal=',')

lista_Q = dados['Qi(t)'].tolist()
lista_H = dados['H(t)'].tolist()

# Número de amostras
N = 10

Q = Vetor(N)
H = Vetor(N + 1)

for i in range(N):
    Q.inserir(i, float(lista_Q[i]))
for i in range(N + 1):
    H.inserir(i, float(lista_H[i]))

# Montagem das matrizes Phi (2 colunas) e Y (1 coluna) com Vetor
Phi_coluna1 = Vetor(N)  # H(k)
Phi_coluna2 = Vetor(N)  # Q(k)
Y = Vetor(N)            # H(k+1)

for i in range(N):
    Phi_coluna1.inserir(i, H.obter(i))
    Phi_coluna2.inserir(i, Q.obter(i))
    Y.inserir(i, H.obter(i + 1))

# Aqui só as matrizes prontas, SEM cálculo de theta (parâmetros)
print("\nMatriz H(k+1) :")
for i in range(N):
    print([round(Y.obter(i), 3)])

print("\nMatriz Φ (Phi) = [H(k), Q(k)]:")
for i in range(N):
    print([round(Phi_coluna1.obter(i), 3), round(Phi_coluna2.obter(i), 3)])



Matriz H(k+1) :
[10.0]
[10.0]
[10.0]
[10.0]
[9.898]
[9.8]
[9.705]
[9.613]
[9.525]
[9.44]

Matriz Φ (Phi) = [H(k), Q(k)]:
[10.0, 27.069]
[10.0, 27.069]
[10.0, 27.069]
[10.0, 27.069]
[10.0, 24.362]
[9.898, 24.362]
[9.8, 24.362]
[9.705, 24.362]
[9.613, 24.362]
[9.525, 24.362]


In [11]:
#@title Equação ARX Genérica
import sympy as sp

# Definindo símbolos para os parâmetros θ
teta1, teta2 = sp.symbols('θ₁ θ₂')
teta_simbolico = sp.Matrix([[teta1], [teta2]])  # vetor coluna

# Símbolos para as matrizes genéricas
Phi_simb = sp.MatrixSymbol('Φ', N, 2)       # matriz Φ (N x 2)
Y_simb = sp.MatrixSymbol('H(k+1)', N, 1)    # vetor H(k+1) (N x 1)

print("== Forma simbólica da equação ARX ==")
display(sp.Eq(Y_simb, Phi_simb * teta_simbolico))

print("\nVetor de parâmetros θ (vertical):")
display(teta_simbolico)


== Forma simbólica da equação ARX ==


Eq(H(k+1), Φ*Matrix([
[θ₁],
[θ₂]]))


Vetor de parâmetros θ (vertical):


Matrix([
[θ₁],
[θ₂]])

In [10]:
#@title Calculo dos Valores de θ pelo Métdodo dos Mínimos Quadrados
from scipy.optimize import minimize

# Função de erro quadrático total: ||Y - Φ * θ||²
def erro_quadratico(teta):
    erro_total = 0.0
    for i in range(N):
        h_estimado = (
            Phi_coluna1.obter(i) * teta[0] +
            Phi_coluna2.obter(i) * teta[1]
        )
        erro = Y.obter(i) - h_estimado
        erro_total += erro**2
    return erro_total

# Estimativa inicial
teta_inicial = [0.0, 0.0]

# Minimização
resultado = minimize(erro_quadratico, teta_inicial)
teta_min = resultado.x

# Criando Vetor θ estimado
teta_estimado = Vetor(2)
teta_estimado.inserir(0, float(teta_min[0]))
teta_estimado.inserir(1, float(teta_min[1]))

# Calculando H(k+1) estimado
H_estimado = Vetor(N)
for i in range(N):
    valor_estimado = (
        Phi_coluna1.obter(i) * teta_estimado.obter(0) +
        Phi_coluna2.obter(i) * teta_estimado.obter(1)
    )
    H_estimado.inserir(i, float(valor_estimado))

print("== Estimativa de θ  via minimize ==")
for i in range(2):
    print([round(float(teta_estimado.obter(i)), 3)])

print("\n== [H(k+1) estimado] = Φ * θ ==")
for i in range(N):
    print([round(float(H_estimado.obter(i)), 3)])


== Estimativa de θ  via minimize ==
[0.885]
[0.042]

== [H(k+1) estimado] = Φ * θ ==
[9.996]
[9.996]
[9.996]
[9.996]
[9.881]
[9.791]
[9.704]
[9.62]
[9.539]
[9.461]
